In [1]:
!git clone "https://github.com/convman/Multimodal-MOSEI"

Cloning into 'Multimodal-MOSEI'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 33 (delta 14), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (33/33), done.


In [2]:
cd Multimodal-MOSEI/data/

/content/Multimodal-MOSEI/data


In [0]:
!chmod +x mosei.sh
!./mosei.sh

In [4]:
cd labels

/content/Multimodal-MOSEI/data/labels


In [0]:
import numpy as np
import h5py
import pandas as pd

In [0]:
Test_labels_surprise = pd.read_csv("mosi2uni_Test_labels_surprise.csv",header=None)
Test_labels_angry = pd.read_csv("mosi2uni_Test_labels_angry.csv",header=None)
Test_labels_disgust = pd.read_csv("mosi2uni_Test_labels_disgust.csv",header=None)
Test_labels_fear = pd.read_csv("mosi2uni_Test_labels_fear.csv",header=None)
Test_labels_happy = pd.read_csv("mosi2uni_Test_labels_happy.csv",header=None)
Test_labels_sad = pd.read_csv("mosi2uni_Test_labels_sad.csv",header=None)

In [0]:
Train_labels_surprise = pd.read_csv("mosi2uni_Train_labels_surprise.csv",header=None)
Train_labels_angry = pd.read_csv("mosi2uni_Train_labels_angry.csv",header=None)
Train_labels_disgust = pd.read_csv("mosi2uni_Train_labels_disgust.csv",header=None)
Train_labels_fear = pd.read_csv("mosi2uni_Train_labels_fear.csv",header=None)
Train_labels_happy = pd.read_csv("mosi2uni_Train_labels_happy.csv",header=None)
Train_labels_sad = pd.read_csv("mosi2uni_Train_labels_sad.csv",header=None)

In [8]:
print(Test_labels_happy.shape)
print(Train_labels_happy.shape)

(4832, 1)
(15290, 1)


In [9]:
video_train = h5py.File("../data/video_train.h5","r")
print(list(video_train.keys()))
video_train = np.array(video_train.get('d1'))
# text_train_emb = text_train_emb[0:15289]
# text_train_emb = text_train_emb[1:]
video_train.shape

['d1']


(15290, 20, 35)

In [10]:
video_test = h5py.File("../data/video_test.h5","r")
print(list(video_test.keys()))
video_test = np.array(video_test.get('d1'))
# text_train_emb = text_train_emb[0:15289]
# text_train_emb = text_train_emb[1:]
video_test.shape

['d1']


(4832, 20, 35)

In [11]:
import keras
from keras.models import load_model
from keras.models import Model,Sequential,Model
from keras.layers import LSTM,Dense,Input
from keras import callbacks
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [0]:
i = Input(shape=(20,35))
x = LSTM(units=256,return_sequences=True)(i)
x = LSTM(units=256,return_sequences=True)(x)
x = LSTM(units=128,return_sequences=True)(x)
x = LSTM(units=128,return_sequences=True)(x)
x = LSTM(units=64,return_sequences=True)(x)
x = LSTM(units=64,return_sequences=True)(x)
x = LSTM(units=32,return_sequences=True)(x)
x = LSTM(units=32,return_sequences=False)(x)
x = Dense(units=32,activation='relu')(x)
x = Dense(units=32,activation='relu')(x)
o = Dense(1,activation='sigmoid')(x)

In [0]:
model_rnn_happy = Model(inputs=i,outputs=o)
model_rnn_angry = Model(inputs=i,outputs=o)
model_rnn_fear = Model(inputs=i,outputs=o)
model_rnn_surprise = Model(inputs=i,outputs=o)
model_rnn_disgust = Model(inputs=i,outputs=o)
model_rnn_sad = Model(inputs=i,outputs=o)

In [20]:
model_rnn_happy.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 20, 35)            0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 20, 256)           299008    
_________________________________________________________________
lstm_10 (LSTM)               (None, 20, 256)           525312    
_________________________________________________________________
lstm_11 (LSTM)               (None, 20, 128)           197120    
_________________________________________________________________
lstm_12 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
lstm_13 (LSTM)               (None, 20, 64)            49408     
_________________________________________________________________
lstm_14 (LSTM)               (None, 20, 64)            33024     
__________

In [0]:
# cd data/

In [26]:
ls

data/  labels/  modify_labels.ipynb  MOSEI_dataset.ipynb  mosei.sh*


In [27]:
model_rnn_happy.compile('adam','binary_crossentropy',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss',mode='min' ,patience=5, min_delta=0.0001,verbose=1)
mcp = ModelCheckpoint("weights_rnn_happy.h5",monitor='val_loss',verbose=1)
history_rnn_happy = model_rnn_happy.fit(video_train,Train_labels_happy,64,20,validation_split=0.1,callbacks=[es, mcp])
print("Evaluating the model=====================================================")
model_rnn_happy.evaluate(video_test,Test_labels_happy)
print("Saving the model=========================================================")
saved_rnn_happy = load_model('weights_rnn_happy.h5')
print("Evaluating saved model===================================================")
saved_rnn_happy.evaluate(video_test,Test_labels_happy)

W0620 08:38:02.686558 140199368533888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 58s 4ms/step - loss: 0.6663 - acc: 0.6116 - val_loss: 0.6822 - val_acc: 0.6030

Epoch 00001: saving model to weights_rnn_happy.h5
Epoch 2/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.6503 - acc: 0.6318 - val_loss: 0.6642 - val_acc: 0.6050

Epoch 00002: saving model to weights_rnn_happy.h5
Epoch 3/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.6379 - acc: 0.6460 - val_loss: 0.6549 - val_acc: 0.6154

Epoch 00003: saving model to weights_rnn_happy.h5
Epoch 4/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.6314 - acc: 0.6493 - val_loss: 0.6605 - val_acc: 0.6187

Epoch 00004: saving model to weights_rnn_happy.h5
Epoch 5/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.6224 - acc: 0.6587 - val_loss: 0.6514 - val_acc: 0.6207

Epoch 00005: saving model to weights_rnn_happy.h5
Epoch 6/20


[0.6223508188661361, 0.6504552980132451]

In [28]:
model_rnn_angry.compile('adam','binary_crossentropy',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss',mode='min' ,patience=5, min_delta=0.0001,verbose=1)
mcp = ModelCheckpoint("weights_rnn_angry.h5",monitor='val_loss',verbose=1)
history_rnn_angry = model_rnn_angry.fit(video_train,Train_labels_angry,64,20,validation_split=0.1,callbacks=[es, mcp])
print("Evaluating the model=====================================================")
model_rnn_angry.evaluate(video_test,Test_labels_angry)
print("Saving the model=========================================================")
saved_rnn_angry = load_model('weights_rnn_angry.h5')
print("Evaluating saved model===================================================")
saved_rnn_angry.evaluate(video_test,Test_labels_angry)

Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 57s 4ms/step - loss: 0.5423 - acc: 0.7651 - val_loss: 0.4799 - val_acc: 0.8234

Epoch 00001: saving model to weights_rnn_angry.h5
Epoch 2/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.5217 - acc: 0.7694 - val_loss: 0.4820 - val_acc: 0.8234

Epoch 00002: saving model to weights_rnn_angry.h5
Epoch 3/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.5327 - acc: 0.7694 - val_loss: 0.4703 - val_acc: 0.8234

Epoch 00003: saving model to weights_rnn_angry.h5
Epoch 4/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.5193 - acc: 0.7694 - val_loss: 0.4775 - val_acc: 0.8234

Epoch 00004: saving model to weights_rnn_angry.h5
Epoch 5/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.5151 - acc: 0.7694 - val_loss: 0.4819 - val_acc: 0.8234

Epoch 00005: saving model to weights_rnn_angry.h5
Epoch 6/20


[0.48475418600025555, 0.7990480132450332]

In [29]:
model_rnn_sad.compile('adam','binary_crossentropy',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss',mode='min' ,patience=5, min_delta=0.0001,verbose=1)
mcp = ModelCheckpoint("weights_rnn_sad.h5",monitor='val_loss',verbose=1)
history_rnn_sad = model_rnn_sad.fit(video_train,Train_labels_sad,64,20,validation_split=0.1,callbacks=[es, mcp])
print("Evaluating the model=====================================================")
model_rnn_sad.evaluate(video_test,Test_labels_sad)
print("Saving the model=========================================================")
saved_rnn_sad = load_model('weights_rnn_sad.h5')
print("Evaluating saved model===================================================")
saved_rnn_sad.evaluate(video_test,Test_labels_sad)

Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 59s 4ms/step - loss: 0.5574 - acc: 0.7425 - val_loss: 0.5268 - val_acc: 0.7626

Epoch 00001: saving model to weights_rnn_sad.h5
Epoch 2/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.5490 - acc: 0.7425 - val_loss: 0.5285 - val_acc: 0.7626

Epoch 00002: saving model to weights_rnn_sad.h5
Epoch 3/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.5457 - acc: 0.7425 - val_loss: 0.5266 - val_acc: 0.7626

Epoch 00003: saving model to weights_rnn_sad.h5
Epoch 4/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.5420 - acc: 0.7425 - val_loss: 0.5332 - val_acc: 0.7626

Epoch 00004: saving model to weights_rnn_sad.h5
Epoch 5/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.5357 - acc: 0.7425 - val_loss: 0.5437 - val_acc: 0.7626

Epoch 00005: saving model to weights_rnn_sad.h5
Epoch 6/20
13761/1376

[0.5944412158222387, 0.6928807947019867]

In [30]:
model_rnn_disgust.compile('adam','binary_crossentropy',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss',mode='min' ,patience=5, min_delta=0.0001,verbose=1)
mcp = ModelCheckpoint("weights_rnn_disgust.h5",monitor='val_loss',verbose=1)
history_rnn_disgust = model_rnn_disgust.fit(video_train,Train_labels_disgust,64,20,validation_split=0.1,callbacks=[es, mcp])
print("Evaluating the model=====================================================")
model_rnn_disgust.evaluate(video_test,Test_labels_disgust)
print("Saving the model=========================================================")
saved_rnn_disgust = load_model('weights_rnn_disgust.h5')
print("Evaluating saved model===================================================")
saved_rnn_disgust.evaluate(video_test,Test_labels_disgust)

Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 60s 4ms/step - loss: 0.4241 - acc: 0.8181 - val_loss: 0.3832 - val_acc: 0.8489

Epoch 00001: saving model to weights_rnn_disgust.h5
Epoch 2/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.4146 - acc: 0.8191 - val_loss: 0.3900 - val_acc: 0.8489

Epoch 00002: saving model to weights_rnn_disgust.h5
Epoch 3/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.4100 - acc: 0.8191 - val_loss: 0.3906 - val_acc: 0.8489

Epoch 00003: saving model to weights_rnn_disgust.h5
Epoch 4/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.4073 - acc: 0.8191 - val_loss: 0.3866 - val_acc: 0.8489

Epoch 00004: saving model to weights_rnn_disgust.h5
Epoch 5/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.4008 - acc: 0.8191 - val_loss: 0.4014 - val_acc: 0.8489

Epoch 00005: saving model to weights_rnn_disgust.h5
E

[0.4348587765796295, 0.8091887417218543]

In [31]:
model_rnn_fear.compile('adam','binary_crossentropy',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss',mode='min' ,patience=5, min_delta=0.0001,verbose=1)
mcp = ModelCheckpoint("weights_rnn_fear.h5",monitor='val_loss',verbose=1)
history_rnn_fear = model_rnn_fear.fit(video_train,Train_labels_fear,64,20,validation_split=0.1,callbacks=[es, mcp])
print("Evaluating the model=====================================================")
model_rnn_fear.evaluate(video_test,Test_labels_fear)
print("Saving the model=========================================================")
saved_rnn_fear = load_model('weights_rnn_fear.h5')
print("Evaluating saved model===================================================")
saved_rnn_fear.evaluate(video_test,Test_labels_fear)

Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 63s 5ms/step - loss: 0.2929 - acc: 0.9140 - val_loss: 0.3000 - val_acc: 0.9111

Epoch 00001: saving model to weights_rnn_fear.h5
Epoch 2/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.2714 - acc: 0.9140 - val_loss: 0.3018 - val_acc: 0.9111

Epoch 00002: saving model to weights_rnn_fear.h5
Epoch 3/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.2640 - acc: 0.9140 - val_loss: 0.3028 - val_acc: 0.9111

Epoch 00003: saving model to weights_rnn_fear.h5
Epoch 4/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.2558 - acc: 0.9140 - val_loss: 0.3227 - val_acc: 0.9111

Epoch 00004: saving model to weights_rnn_fear.h5
Epoch 5/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.2504 - acc: 0.9142 - val_loss: 0.3163 - val_acc: 0.9111

Epoch 00005: saving model to weights_rnn_fear.h5
Epoch 6/20
13761

[0.24568041919773778, 0.929635761589404]

In [32]:
model_rnn_surprise.compile('adam','binary_crossentropy',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss',mode='min' ,patience=5, min_delta=0.0001,verbose=1)
mcp = ModelCheckpoint("weights_rnn_surprise.h5",monitor='val_loss',verbose=1)
history_rnn_surprise = model_rnn_surprise.fit(video_train,Train_labels_surprise,64,20,validation_split=0.1,callbacks=[es, mcp])
print("Evaluating the model=====================================================")
model_rnn_surprise.evaluate(video_test,Test_labels_surprise)
print("Saving the model=========================================================")
saved_rnn_surprise = load_model('weights_rnn_surprise.h5')
print("Evaluating saved model===================================================")
saved_rnn_surprise.evaluate(video_test,Test_labels_surprise)

Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 65s 5ms/step - loss: 0.3323 - acc: 0.8966 - val_loss: 0.3016 - val_acc: 0.9091

Epoch 00001: saving model to weights_rnn_surprise.h5
Epoch 2/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.3256 - acc: 0.8966 - val_loss: 0.3009 - val_acc: 0.9091

Epoch 00002: saving model to weights_rnn_surprise.h5
Epoch 3/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.3256 - acc: 0.8966 - val_loss: 0.3109 - val_acc: 0.9091

Epoch 00003: saving model to weights_rnn_surprise.h5
Epoch 4/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.3213 - acc: 0.8966 - val_loss: 0.3081 - val_acc: 0.9084

Epoch 00004: saving model to weights_rnn_surprise.h5
Epoch 5/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.3191 - acc: 0.8983 - val_loss: 0.3091 - val_acc: 0.9071

Epoch 00005: saving model to weights_rnn_surprise

[0.3484039660419849, 0.8845198675496688]

In [33]:
ls

data/                mosei.sh*               weights_rnn_happy.h5
labels/              weights_rnn_angry.h5    weights_rnn_sad.h5
modify_labels.ipynb  weights_rnn_disgust.h5  weights_rnn_surprise.h5
MOSEI_dataset.ipynb  weights_rnn_fear.h5


In [0]:
from google.colab import files
files.download('weights_rnn_angry.h5')
files.download('weights_rnn_disgust.h5')
files.download('weights_rnn_fear.h5')
files.download('weights_rnn_happy.h5')
files.download('weights_rnn_sad.h5')
files.download('weights_rnn_surprise.h5')